In [72]:
def ComputePlayerPIE(Id, season = '2021', seasonType = 'Regular Season'):
    from nba_api.stats.endpoints import boxscoreadvancedv2, playergamelog
    import numpy as np
    import time
    
    player_log_df = playergamelog.PlayerGameLog(player_id = Id , season = season, 
                                           season_type_all_star = seasonType).get_data_frames()[0]
    
    if not player_log_df.empty:
        game_ids = player_log_df["Game_ID"].to_numpy()
        player_season_PIE = []
        team_season_PIE = []

        for ID in game_ids:
            time.sleep(1)
            career = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = ID)
            player_boxscore = career.get_data_frames()[0]
            player_id_array = player_boxscore['PLAYER_ID'].to_numpy()
            player_row = np.where(player_id_array == int(Id))[0][0]
            player_box = player_boxscore.iloc[[player_row]]
            player_game_pie = player_box['PIE'][player_row]
            player_season_PIE.append(player_game_pie)

            team_boxscore = career.get_data_frames()[1]
            team_id_array = team_boxscore['TEAM_ID']
            team_id = player_box['TEAM_ID'][player_row]
            team_row = np.where(team_id_array == team_id)[0][0]
            team_box = team_boxscore.iloc[[team_row]]
            team_game_pie = team_box['PIE'][team_row]
            team_season_PIE.append(team_game_pie)

        #print(player_season_PIE)
        player_PIE = sum(player_season_PIE) / len(player_season_PIE)
        team_PIE = sum(team_season_PIE) / len(team_season_PIE)
    else:
        player_PIE = 0
        team_PIE = 0
    
    return player_PIE, team_PIE

In [54]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import commonplayerinfo
#just change the file path
physical_attributes = pd.read_csv('/Users/Tim/Documents/GitHub/NBA-Draft-Biola-Summer-Research/draft_data_2000_2016_no_empty_cells.csv')

#display(physical_attributes)

player_ids = physical_attributes["PLAYER_ID"].to_numpy()
draft_class = np.add(physical_attributes["DRAFT_CLASS"].to_numpy(),5).astype(str)
player_pies = []

#print(draft_class)
#player_ids_int = player_ids.astype(int)

for pid, draft_year in zip(player_ids, draft_class):
    
    pid_pie = ComputePlayerPIE(pid,draft_year)[0]
    player_pies.append(pid_pie)
    
data_dict = {}
name1 = PIE_SCORES
name2 = PLAYER_ID

data_dict[name2] = player_ids
data_dict[name1] = player_pies
#print(player_pies)
df = pd.DataFrame(data = data_dict)

df.to_csv('draft_data_5_year_pie_score.csv', index=False)
#display(df)

['2005' '2005' '2005' ... '2021' '2021' '2021']
